In [308]:
#Invite everyone to the data science par-tay

import pandas as pd

from ipywidgets import widgets

# Predicting suicide rates based on supposed risk factors

Mental health is a prevalent problem in today's world. Being able to predict the percentage of a population in a region who will commit suicide could lead to more money for support being put in place in those regions and for those most at risk.

I have looked at some of the risk factors for suicide listed on mentalhealth.org (https://www.mentalhealth.org.uk/a-to-z/s/suicide) and have chosen to investigate whether unemployment, alcohol consumption, poverty, location and murder rates are able to predict suicide rates. I will take data between 2000 and 2016 one because that keeps the predictions relevant to this year and 2) because lots of data from before these years is missing. 

# Gathering and preprocessing the data

In [459]:
#Load all of the data
unemployment_percentage = pd.read_csv("long_term_unemployment_rate_percent (1).csv")
poverty_percentage = pd.read_csv("alternative_poverty_percent_below_nationally_defined_poverty.csv")
murders_per_100000 = pd.read_csv("murder_per_100000_people.csv")
alcohol_per_capita = pd.read_csv("data.csv")
suicide_per_1000 = pd.read_csv("suicide_per_100000_people (1).csv")

In [402]:
#Have a look at some of the data

unemployment_percentage.head()

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.50,4.910,5.67,5.28,5.30,5.53,5.76,6.09,NaN,NaN
1,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.15,NaN,NaN,NaN
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.56,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.88,0.925,1.22,1.11,1.28,1.04,1.22,NaN,NaN,1.14
4,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.690,6.77,6.46,5.80,NaN,7.15,7.11,5.4,NaN


In [430]:
#The alcohol datasets were taken from a different source and so are formatted differently. I'll play around with the 
#csv file to get it into the same format as the others to make things run more smoothly. 
alcohol_per_capita.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,"Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol).1","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol).2","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol).3","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol).4","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol).5","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol).6"
0,Country,Data Source,Beverage Types,2016.00,2015.00,2014.00,2013.00,2012.00,2011.00,2010.00
1,Afghanistan,Data source,All types,NaN,0.02,0.03,0.03,0.04,0.04,0.03
2,Albania,Data source,All types,5.07,4.77,4.81,5.06,5.43,5.65,5.53
3,Algeria,Data source,All types,0.56,0.56,0.56,0.54,0.49,0.44,0.39
4,Andorra,Data source,All types,10.06,9.97,9.95,9.78,10.06,10.31,10.64


In [460]:
#Drop unecessary columns
#Adjust the headers to match other datasets
#Drop the first row (now a duplicate of header)

alcohol_per_capita = alcohol_per_capita.drop(["Unnamed: 1", "Unnamed: 2"], axis = "columns")

headers = ["country"]
for header_names in alcohol_per_capita.iloc[0][1:]:
    headers.append(str(int(header_names)))
    
alcohol_per_capita.iloc[0] = headers
alcohol_per_capita.columns = alcohol_per_capita.iloc[0]
alcohol_per_capita = alcohol_per_capita.drop(alcohol_per_capita.index[0])

In [461]:
#Check everything looks dandy
alcohol_per_capita.head()

,country,2016,2015,2014,2013,2012,2011,2010
1,Afghanistan,NaN,0.02,0.03,0.03,0.04,0.04,0.03
2,Albania,5.07,4.77,4.81,5.06,5.43,5.65,5.53
3,Algeria,0.56,0.56,0.56,0.54,0.49,0.44,0.39
4,Andorra,10.06,9.97,9.95,9.78,10.06,10.31,10.64
5,Angola,4.7,5.65,9,8.02,8.14,7.86,7.67


In [407]:
# Only keep years after 2000 from the databases

less_than_2000_column_indexes = []

def get_pre_2000_indexes(data):
    less_than_2000_column_indexes = []
    for column_name in data.columns.values[1:]: #don't include 'country' column
        if (int(column_name) < 2000):
            less_than_2000_column_indexes.append(data.columns.get_loc(column_name))
    return less_than_2000_column_indexes

get_pre_2000_indexes(unemployment_percentage)
unemployment_percentage = unemployment_percentage.drop(unemployment_percentage.columns[get_pre_2000_indexes(unemployment_percentage)], axis="columns")

get_pre_2000_indexes(poverty_percentage)
poverty_percentage = poverty_percentage.drop(poverty_percentage.columns[get_pre_2000_indexes(poverty_percentage)], axis="columns")

get_pre_2000_indexes(murders_per_100000)
murders_per_100000 = murders_per_100000.drop(murders_per_100000.columns[get_pre_2000_indexes(murders_per_100000)], axis="columns")

get_pre_2000_indexes(suicide_per_1000)
suicide_per_1000 = suicide_per_1000.drop(suicide_per_1000.columns[get_pre_2000_indexes(suicide_per_1000)], axis="columns")


Looking at all the data, it seems as though all of the dataframes are in the same format with the rows sorted by countries and columns as the years so I will need to re-arrange them all and combine them. However, before I do this, I am going to look at the mising values of each data files. I predict that certain countries will have lots of missing data rather than some years having missing data for lots of countries. I will investigate this and if this is the case, I will drop the countries with a substantial amount of missing data. 

In [307]:
poverty_percentage.head()

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.3,NaN,NaN,NaN,35.8,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,NaN,NaN,25.4,NaN,NaN,18.5,NaN,NaN,12.4,NaN,NaN,NaN,14.3,NaN,NaN,NaN,NaN,NaN
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,NaN,NaN,NaN,NaN,NaN
3,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.3,NaN


## Sort out the missing values in each dataset before combining

In [312]:
def missing_values_columns(data): 
    number_of_missing_values = data.isnull().sum()
    percentage_missing = (data.isnull().sum())/data.isnull().count()
    missing_values_table = pd.concat([percentage_missing, number_of_missing_values], axis=1, keys=["Percentage missing", "Number of missing values"])
    return missing_values_table.sort_values("Percentage missing", ascending=False)

def missing_values_rows(data): 
    number_of_missing_values = data.isnull().sum(axis="columns")
    percentage_missing = (data.isnull().sum(axis="columns"))/data.isnull().count(axis="columns")
    missing_values_table = pd.concat([percentage_missing, number_of_missing_values], axis=1, keys=["Percentage missing", "Number of missing values"])
    return missing_values_table.sort_values("Percentage missing", ascending=False)

As predicted, some countries have a substantial amount of missing data and some countries have very little, although when categorised by year, all years have a fair amount of missing data due to lots of countries in each year not submitting the data. I will delete all years with countries with >25% of missing data. 

In [150]:
#Missing data for each year 
missing_values_columns(unemployment_percentage)

,Percentage missing,Number of missing values
2000,0.644628,78
2001,0.619835,75
2002,0.603306,73
2003,0.578512,70
2004,0.570248,69
2005,0.537190,65
2007,0.520661,63
2006,0.512397,62
2008,0.479339,58
2017,0.471074,57


In [151]:
#Missing data for each country 
missing_values_rows(unemployment_percentage)

,Percentage missing,Number of missing values
120,0.894737,17
17,0.894737,17
70,0.894737,17
73,0.894737,17
37,0.894737,17
76,0.894737,17
61,0.894737,17
90,0.894737,17
1,0.894737,17
92,0.894737,17


In [152]:
unemployment_percentage = pd.read_csv("long_term_unemployment_rate_percent (1).csv")

unemployment_original_size = unemployment_percentage.size

unemployment_percentage = unemployment_percentage.dropna(thresh=0.5*len(unemployment_percentage), axis="columns")
unemployment_percentage = unemployment_percentage.transpose()
unemployment_percentage = unemployment_percentage.dropna(thresh=0.5*len(unemployment_percentage), axis="columns")

In [153]:
print("Unemployment orignally included {0} countries and after dropping those with a high percentage of missing values it contains {1} countries".format(unemployment_original_size, unemployment_percentage.size))

Unemployment orignally included 3509 countries and after dropping those with a high percentage of missing values it contains 836 countries


In [154]:
missing_values_rows(unemployment_percentage)

,Percentage missing,Number of missing values
2017,0.236842,18
2008,0.210526,16
2015,0.065789,5
2016,0.052632,4
2009,0.039474,3
2010,0.039474,3
2014,0.039474,3
2012,0.026316,2
2013,0.026316,2
2011,0.013158,1


In [155]:
#urban poverty reduce missing values 
#urban_poverty_percentage = urban_poverty_percentage.dropna(thresh=0.1*len(urban_poverty_percentage), axis="columns")
#urban_poverty_percentage = urban_poverty_percentage.transpose()
#urban_poverty_percentage = urban_poverty_percentage.dropna(thresh=0.6*len(urban_poverty_percentage), axis="columns")

#missing_values_columns(urban_poverty_percentage)

In [156]:
#Repeat with other dataframes
#Transpose and drop those with >25% missing data

poverty_percentage = poverty_percentage.transpose()
poverty_percentage = poverty_percentage.dropna(thresh=0.5*len(poverty_percentage), axis="columns")

murders_per_100000 = murders_per_100000.transpose()
murders_per_100000 = murders_per_100000.dropna(thresh=0.5*len(murders_per_100000), axis="columns")

#alcohol_per_capita = alcohol_per_capita.transpose()
#alcohol_per_capita = alcohol_per_capita.dropna(thresh=0.3*len(alcohol_per_capita), axis="columns")

suicide_per_1000 = suicide_per_1000.transpose()
suicide_per_1000 = suicide_per_1000.dropna(thresh=0.5*len(suicide_per_1000), axis="columns")


## Combining the dataframes

I want the dataframes to be able to combine to be in the format:
year - country - rest of variables

so I need to melt each dataframe so that it is of the form 
year - country - variable

and then I will be able to combine them. 

In [314]:
unemployment_percentage = unemployment_percentage.transpose()
unemployment_percentage = pd.melt(unemployment_percentage, id_vars=['country']).rename(columns={'variable':'year','value': "unemployment_percentage"}).sort_values(by=["country", "year"]).reset_index().drop("index", axis="columns")


In [158]:
unemployment_percentage.head()

,country,year,unemployment_percentage
0,Albania,2008,4.5
1,Albania,2009,4.91
2,Albania,2010,5.67
3,Albania,2011,5.28
4,Albania,2012,5.3


In [316]:
#Melting the rest of the dataframes

poverty_percentage = poverty_percentage.transpose()
murders_per_100000 = murders_per_100000.transpose()
#alcohol_per_capita = alcohol_per_capita.transpose()
suicide_per_1000 = suicide_per_1000.transpose()

poverty_percentage = pd.melt(poverty_percentage, id_vars=['country']).rename(columns={'variable':'year','value': "rural_poverty_percentage"}).sort_values(by=["country", "year"]).reset_index().drop("index", axis="columns")
murders_per_100000 = pd.melt(murders_per_100000, id_vars=['country']).rename(columns={'variable':'year','value': "murders_per_100000"}).sort_values(by=["country", "year"]).reset_index().drop("index", axis="columns")
#alcohol_per_capita = pd.melt(alcohol_per_capita, id_vars=['country']).rename(columns={'variable':'year','value': "alcohol_per_capita"}).sort_values(by=["country", "year"]).reset_index().drop("index", axis="columns")
suicide_per_1000= pd.melt(suicide_per_1000, id_vars=['country']).rename(columns={'variable':'year','value': "suicide_per_1000"}).sort_values(by=["country", "year"]).reset_index().drop("index", axis="columns")
    
    

In [160]:
poverty_percentage.tail()

,country,year,rural_poverty_percentage
259,Venezuela,2013,29.4
260,Venezuela,2014,29.5
261,Venezuela,2015,33.1
262,Venezuela,2016,NaN
263,Venezuela,2017,NaN


In [317]:
#Combine the dataframes where country and year are the same

merge1 = pd.merge(unemployment_percentage, poverty_percentage, on=["country", "year"], how='outer').sort_values(by=["country", "year"])
merge2 = pd.merge(merge1, murders_per_100000, on=["country", "year"], how='outer').sort_values(by=["country", "year"])
#merge3 = pd.merge(merge2, alcohol_per_capita, on=["country", "year"], how='outer').sort_values(by=["country", "year"])
full_data = pd.merge(merge2, suicide_per_1000, on=["country", "year"], how='outer').sort_values(by=["country", "year"])


In [318]:
#Check everything is successfully merged

full_data.head()

,country,year,unemployment_percentage,rural_poverty_percentage,murders_per_100000,suicide_per_1000
0,Afghanistan,2000,NaN,NaN,NaN,NaN
1,Afghanistan,2001,NaN,NaN,NaN,NaN
2,Afghanistan,2002,NaN,NaN,NaN,NaN
3,Afghanistan,2003,NaN,NaN,NaN,NaN
4,Afghanistan,2004,NaN,NaN,NaN,NaN


In [262]:
#Check that the size of the dataset looks reasonable. 

print("The length of the unemployment dataset was {0}, the length of the suicide dataset was {1} and the length of the full dataset is {2}. This seems reasonable.".format(len(unemployment_percentage), len(suicide_per_1000), len(full_data)))

The length of the unemployment dataset was 3388, the length of the suicide dataset was 6968 and the length of the full dataset is 9746. This seems reasonable.


# Look at missing data

In [319]:
def missing_values(data): 
    number_of_missing_values = data.isnull().sum()
    percentage_missing = (data.isnull().sum())/data.isnull().count()
    missing_values_table = pd.concat([percentage_missing, number_of_missing_values], axis=1, keys=["Percentage missing", "Number of missing values"])
    return missing_values_table.sort_values("Percentage missing", ascending=False)

In [372]:
full_data_copy = full_data_copy.transpose()

In [373]:
full_data_copy.head()

,country,year,unemployment_percentage,rural_poverty_percentage,murders_per_100000,suicide_per_1000
18,Albania,2000,NaN,NaN,3.96,1.88
19,Albania,2001,NaN,NaN,7.58,4.04
20,Albania,2002,NaN,25.4,7.4,4.53
21,Albania,2003,NaN,NaN,5.53,4.08
22,Albania,2004,NaN,NaN,4.29,4.72


In [367]:
full_data_copy = full_data.copy()
full_data_copy = full_data_copy.dropna(thresh=0.55*len(full_data_copy), axis="columns")


In [368]:
missing_values_rows(full_data_copy)

,Percentage missing,Number of missing values
rural_poverty_percentage,0.578171,588
murders_per_100000,0.225172,229
suicide_per_1000,0.221239,225
unemployment_percentage,0.138643,141
country,0.000000,0
year,0.000000,0


In [369]:
len(full_data_copy.columns)

1017